Run the beds live sql and then wrangle the data into a simple census

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import sqlalchemy as sa

Assume that we're running from the notebooks directory and need to pull data from ./src/mock/mock.db

In [ ]:
sqlite_db = "../src/mock/mock.db"
assert Path(sqlite_db).is_file()

In [ ]:
engine = sa.create_engine(f"sqlite:///{sqlite_db}")

In [ ]:
df = pd.read_sql("bedsmock", engine)

In [ ]:
df.iloc[0]

So now you want to return a dataframe with the following characteristics
- per department (one row per department)
- count number of beds
- count number of occupied beds
- count number of empty beds
- last date of closed discharge from that department (i.e. if > 1 week and most beds empty then is the department closed)
- number of side rooms?

In [ ]:
temp = (
    df["location_string"]
    .str.split("^", expand=True)
    .rename(columns={0: "dept", 1: "room", 2: "bed"})
)
for s in ["dept", "room", "bed"]:
    df[s] = temp[s]

In [ ]:
del temp

In [ ]:
mask = df["bed"].str.lower().isin(["null", "wait"])
df = df[~mask]

In [ ]:
df.shape

In [ ]:
groups = df.groupby("department")

In [ ]:
groups.get_group("GWB L01 CRITICAL CARE")

In [ ]:
groups.agg(beds=("location_id", "count"))